In [4]:
# State-of-Charge (SOC) estimation of Panasonic 18650PF Li-ion Battery
# With Hidden Layers
# Data driven estimation using machine learning - neural nets
# data: https://data.mendeley.com/datasets/wykht8y7tg/1
# NN - 25degC
# contributor:
# Author: Heloisa
# FCET - PUC/SP

# Date: 22/08/2023

import tensorflow as tf
from keras.models import Model#, Sequential
from keras import layers#, regularizers, models
from keras import Input
import numpy as np
import matplotlib.pyplot as plt

from google.colab import files
uploaded = files.upload()

import scipy.io
train_data_dic = scipy.io.loadmat('train_data.mat')

train_data = train_data_dic['train_data']

test_data_dic = scipy.io.loadmat('test_data.mat')
test_data = test_data_dic['test_data']

train_label_dic = scipy.io.loadmat('train_label.mat')
train_targets = train_label_dic['SOC_label_train']

test_label_dic = scipy.io.loadmat('test_label.mat')
test_targets = test_label_dic['SOC_label_test']

# This is the Neural Net Model
def build_model():
    # modelo equivalente usando a API funcional
    input_tensor = Input(shape=(train_data.shape[1],))
    x = layers.Dense(n_hidden, activation='relu')(input_tensor) # hidden layer 1
    x = layers.Dense(n_hidden, activation='relu')(x) # hidden layer 2
    x = layers.Dense(n_hidden, activation='relu')(x) # hidden layer 3
    x = layers.Dense(n_hidden, activation='relu')(x) # hidden layer 4
    output_tensor = layers.Dense(1)(x)
    model = Model(input_tensor, output_tensor)
    #model.compile(tf.keras.optimizers.SGD(learning_rate=lr), loss='mse', metrics=['mae'])
    #model.compile(tf.keras.optimizers.RMSprop(learning_rate=lr), loss='mse', metrics=['mae'])
    #model.compile(tf.keras.optimizers.Adamax(learning_rate=lr), loss='mse', metrics=['mae'])
    model.compile(tf.keras.optimizers.Adam(learning_rate=lr), loss='mse', metrics=['mae'])
    #model.compile(tf.keras.optimizers.Adadelta(learning_rate=lr), loss='mse', metrics=['mae'])
    #model.compile(tf.keras.optimizers.Adagrad(learning_rate=lr), loss='mse', metrics=['mae'])
    #model.compile(tf.keras.optimizers.Nadam(learning_rate=lr), loss='mse', metrics=['mae'])
    #model.compile(tf.keras.optimizers.Ftrl(learning_rate=lr), loss='mse', metrics=['mae'])
    return model

# Regularization with K-Fold
k = 4 # number of folds for cross-validation
num_val_samples = len(train_data) // k
num_epochs = 100 # let default value be 50
num_epochs_train = 50 # let default value be 50
all_scores = []
BATCH_SIZE = 128
n_hidden = 256 # units
# Step size
lr = 0.001
# lr = 0.0001

all_loss_histories = []
all_mae_histories = []
all_val_loss_histories = []
all_val_mae_histories = []

from keras import backend as K
# Some memory clean-up
K.clear_session()
# K-fold validation

for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples:]], axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    treino = model.fit(partial_train_data, partial_train_targets, validation_data=(val_data, val_targets),
                       epochs=num_epochs, batch_size=BATCH_SIZE, verbose=1)
    # Keep a record of how well the model did at each epoch
    # save the per-epoch validation score log:
    loss_history = treino.history['loss']
    mae_history = treino.history['mae']
    val_loss_history = treino.history['val_loss']
    val_mae_history = treino.history['val_mae']

    all_loss_histories.append(loss_history)
    all_mae_histories.append(mae_history)
    all_val_loss_histories.append(val_loss_history)
    all_val_mae_histories.append(val_mae_history)

# We can then compute the average of the per-epoch MAE scores for all folds:
average_loss_history = [np.mean([x[i] for x in all_loss_histories]) for i in range(num_epochs)]
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]
average_val_loss_history = [np.mean([x[i] for x in all_val_loss_histories]) for i in range(num_epochs)]
average_val_mae_history = [np.mean([x[i] for x in all_val_mae_histories]) for i in range(num_epochs)]

font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }

# plt.clf()

plt.figure()
plt.plot(range(1, len(average_loss_history) + 1), 20*np.log10(average_loss_history), 'b',label='Loss function (dB)')
plt.plot(range(1, len(average_val_loss_history) + 1), 20*np.log10(average_val_loss_history), 'r', label='Validation loss function (dB)')
plt.title('Validation Phase - Learning curves', fontdict=font)
plt.xlabel('Epoch', fontdict=font)
plt.ylabel('Loss function', fontdict=font)
plt.legend()
plt.show()

plt.figure()
plt.plot(range(1, len(average_mae_history) + 1), 20*np.log10(average_mae_history), 'b', label='MAE (dB)')
plt.plot(range(1, len(average_val_mae_history) + 1), 20*np.log10(average_val_mae_history), 'r', label='Validation MAE (dB)' )
plt.title('Validation Phase - Learning curves', fontdict=font)
plt.xlabel('Epoch', fontdict=font)
plt.ylabel('Error - MAE', fontdict=font)
plt.legend()

plt.show()

# Model construction ... THIS IS IMPORTANT !!!
model = build_model()
# Model summary
model.summary()

# Train the model on the entire training data - REAL training phase !!!
real_train = model.fit(train_data, train_targets, epochs=num_epochs_train, batch_size=128, verbose=1)

# Keep a record of how well the model did at each epoch at REAL training phase
# save the per-epoch validation score log:
loss_hist = real_train.history['loss']
mae_hist = real_train.history['mae']

plt.figure()

plt.plot(range(1, len(loss_hist) + 1), 20*np.log10(loss_hist), 'r', label='loss function (dB)')
plt.plot(range(1, len(mae_hist) + 1),  20*np.log10(mae_hist), 'b', label='MAE (dB)')
plt.title('Training Phase - Learning curves', fontdict=font)
plt.xlabel('Epoch', fontdict=font)
plt.ylabel('MAE/Loss', fontdict=font)
plt.legend()

plt.show()

# Evaluate the model on the test data
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

# Performance on the test data
print('Test MAE score', test_mae_score)
print('Test MSE score', test_mse_score)

test_pred = model.predict(test_data)

plt.figure()

plt.plot(test_targets, test_pred, 'b', label='SOC')
plt.title('SOC: test set vs predictions', fontdict=font)
plt.xlabel('SOC', fontdict=font)
plt.ylabel('Predicted SOC', fontdict=font)
plt.legend()

plt.show()

# 4 hidden layers - 256 hidden units - batch size = 128 - step size = 0.001
# Mean Absolute Error (MAE) and MSE - performance over the TEST SET
# Optimizers

# Date: 22/08/2023

# SGD: MAE = , MSE = 0
# Adagrad: MAE = , MSE =
# RMSProp: MAE = , MSE =
# Adam: MAE = , MSE =

# Adadelta: MAE = , MSE =
# Adamax:  MAE = , MSE =
# Nadam: MAE = , MSE =
# Ftlr: MAE = , MSE =

TypeError: ignored